# PydanticAI Agent

In [1]:
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel, OpenAIResponsesModelSettings
from pydantic_ai.providers.openai import OpenAIProvider

from pydantic_ai.models import ModelRequestParameters
from pydantic_ai.messages import ModelMessage
from pydantic_ai.settings import ModelSettings

from config import settings

# Model Init

OpenAIModel
* https://ai.pydantic.dev/models/#environment-variable
* https://github.com/pydantic/pydantic-ai/blob/1def7df728a5e757ceb16797a8b608e00da70982/pydantic_ai_slim/pydantic_ai/models/openai.py#L146

ModelRequestParameters
* Configuration for an agent's request to a model, specifically related to tools and output handling.


ModelMessage

ModelSettings

In [2]:
provider = OpenAIProvider(
    base_url=settings.llm_base_url,
    api_key=settings.llm_api_key
)
model = OpenAIModel(
    model_name=settings.llm_model,
    provider=provider
)

In [3]:
model_settings = OpenAIResponsesModelSettings()

# Agent Test

In [4]:
from enum import Enum
from typing import List

from pydantic import BaseModel, Extra, Field

In [5]:
class EmotionalState(str, Enum):
    positive="positive"
    neutral="netural"
    negative="negative"
    
class AllowedActions(str, Enum):
    talk="talk"
    walk="walk"
    
class CharacterAction(BaseModel):
    action_type: AllowedActions
    description: str
    
class CharacterState(BaseModel):
    emotional_state: EmotionalState
    description: str
    previous_actions: List[CharacterAction]

In [6]:
CharacterActionPredictionDescription = '''You are a creative storyteller.
Given the character state in the form of
{"emotional_state": positive/neutral/negative, "description": str}

Predict a suitable next action for the character to take
The type of actions that are allowed are the following
* talk
* walk

Character Action is in the form of
{"action_type": str, "description": str}'''

CharacterActionPredictionAgent = Agent(
    model=model,
    name="character_action_predictor",
    deps_type=CharacterState,
    result_type=CharacterAction,
    system_prompt = CharacterActionPredictionDescription
)

In [7]:
state = CharacterState(
    emotional_state=EmotionalState.positive,
    description="character is on a date",
    previous_actions=[]
)

In [8]:
result = await CharacterActionPredictionAgent.run(
    "Predict the next action",
    deps=state
)

In [12]:
result.data.model_dump()

{'action_type': <AllowedActions.talk: 'talk'>,
 'description': 'The character decides to talk to a passerby to ask for directions, hoping to find their way back to the main road after feeling lost in the woods. '}

In [9]:
# # Test model inf
# messages = [{"role": "user", "content": "Explain quantum mechanics in one simple sentence."}]
# response = model(messages)
# print(response)